<img src="../exploratoryAnalysis/zona_jobs.png">

# Trabajo Practico N° 2

### Aqui procederemos a la limpieza de datos

#### 1-1 Carga de datos

In [1]:
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
from matplotlib.mlab import bivariate_normal
%matplotlib inline
import pandas as pd
from dateutil.parser import parse
import numpy as np
import warnings
import datetime as dt
import seaborn as sns
warnings.filterwarnings('ignore')
import plotly

In [ ]:
# Carga de archivos
post_educacion  = pd.read_csv('../data/fiuba_1_postulantes_educacion.csv', dtype = {
    'nombre': 'category',
    'estado': 'category'
})

In [ ]:
post_gen_edad   = pd.read_csv('../data/fiuba_2_postulantes_genero_y_edad.csv', dtype = {
    'sexo': 'category'    
})

In [2]:
vistas          = pd.read_csv('../data/fiuba_3_vistas.csv'       , parse_dates=['timestamp'])

In [9]:
postulaciones   = pd.read_csv('../data/fiuba_4_postulaciones.csv', parse_dates=['fechapostulacion'])

In [15]:
avisos_online   = pd.read_csv('../data/fiuba_5_avisos_online.csv')

In [2]:
avisos_detalles = pd.read_csv('../data/fiuba_6_avisos_detalle.csv', dtype= {
    'nombre_zona': 'category',
    'ciudad': 'category',
    'tipo_de_trabajo': 'category',
    'nivel_laboral': 'category',
})

In [ ]:
# Carga de archivos
post_educacion_pre  = pd.read_csv('../data/pre_abril/fiuba_1_postulantes_educacion.csv', dtype = {
    'nombre': 'category',
    'estado': 'category'
})

In [ ]:
post_gen_edad_pre   = pd.read_csv('../data/pre_abril/fiuba_2_postulantes_genero_y_edad.csv', dtype = {
    'sexo': 'category'    
})

In [3]:
vistas_pre          = pd.read_csv('../data/pre_abril/fiuba_3_vistas.csv'       , parse_dates=['timestamp'])

In [10]:
postulaciones_pre   = pd.read_csv('../data/pre_abril/fiuba_4_postulaciones.csv', parse_dates=['fechapostulacion'])

In [16]:
avisos_online_pre   = pd.read_csv('../data/pre_abril/fiuba_5_avisos_online.csv')

In [3]:
avisos_detalles_pre = pd.read_csv('../data/pre_abril/fiuba_6_avisos_detalle.csv', dtype= {
    'nombre_zona': 'category',
    'ciudad': 'category',
    'tipo_de_trabajo': 'category',
    'nivel_laboral': 'category',
})

In [ ]:
# Carga de archivos
post_educacion_post  = pd.read_csv('../data/post_abril/fiuba_1_postulantes_educacion.csv', dtype = {
    'nombre': 'category',
    'estado': 'category'
})

In [4]:
vistas_post          = pd.read_csv('../data/post_abril/fiuba_3_vistas.csv'       , parse_dates=['timestamp'])

In [4]:
avisos_detalles_post = pd.read_csv('../data/post_abril/fiuba_6_avisos_detalle.csv', dtype= {
    'nombre_zona': 'category',
    'ciudad': 'category',
    'tipo_de_trabajo': 'category',
    'nivel_laboral': 'category',
})

In [5]:
avisos_detalles_last = pd.read_csv('../data/post_abril/fiuba_6_avisos_detalle_last.csv', dtype= {
    'nombre_zona': 'category',
    'ciudad': 'category',
    'tipo_de_trabajo': 'category',
    'nivel_laboral': 'category',
})

## Ahora agrupo todos en un mismo DF

In [ ]:
frames = [ post_educacion, post_educacion_pre, post_educacion_post ]
post_ed_total = pd.concat(frames)
post_ed_total.head()

In [ ]:
frames = [ post_gen_edad, post_gen_edad_pre]
post_gen_edad_total = pd.concat(frames)
post_gen_edad_total.head()

In [5]:
frames = [ vistas, vistas_pre, vistas_post]
vistas_total = pd.concat(frames)
vistas_total.head()

,idAviso,timestamp,idpostulante
0,1111780242,2018-02-23 18:38:13.187,YjVJQ6Z
1,1112263876,2018-02-23 18:38:14.296,BmVpYoR
2,1112327963,2018-02-23 18:38:14.329,wVkBzZd
3,1112318643,2018-02-23 18:38:17.921,OqmP9pv
4,1111903673,2018-02-23 18:38:18.973,DrpbXDP


In [11]:
frames = [ postulaciones, postulaciones_pre]
postulaciones_total = pd.concat(frames)
postulaciones_total.head()

,idaviso,idpostulante,fechapostulacion
0,1112257047,NM5M,2018-01-15 16:22:34
1,1111920714,NM5M,2018-02-06 09:04:50
2,1112346945,NM5M,2018-02-22 09:04:47
3,1112345547,NM5M,2018-02-22 09:04:59
4,1112237522,5awk,2018-01-25 18:55:03


In [17]:
frames = [ avisos_online, avisos_online_pre]
avisos_online_total = pd.concat(frames)
avisos_online_total.head()

,idaviso
0,1112355872
1,1112335374
2,1112374842
3,1111984070
4,1111822480


In [6]:
frames = [ avisos_detalles, avisos_detalles_pre, avisos_detalles_post, avisos_detalles_last]
avisos_detalles_total = pd.concat(frames)
avisos_detalles_total.head()

,idaviso,idpais,titulo,descripcion,nombre_zona,ciudad,mapacalle,tipo_de_trabajo,nivel_laboral,nombre_area,denominacion_empresa
0,8725750,1,VENDEDOR/A PROVINCIA DE SANTA FE,<p><strong><strong>Empresa:</strong></strong> ...,Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Comercial,VENTOR
1,17903700,1,Enfermeras,<p>Solicitamos para importante cadena de farma...,Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Salud,Farmacias Central Oeste
2,1000150677,1,Chofer de taxi,<p>TE GUSTA MANEJAR? QUERES GANAR PLATA HACIEN...,Capital Federal,NaN,Empedrado 2336,Full-time,Senior / Semi-Senior,Transporte,FAMITAX SRL
3,1000610287,1,CHOFER DE CAMIONETA BAHIA BLANCA - PUNTA ALTA,<p><strong>Somos una empresa multinacional que...,Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Transporte,Wurth Argentina S.A
4,1000872556,1,Operarios de Planta - Rubro Electrodomésticos,<p><strong>OPERARIOS DE PLANTA</strong></p><p>...,Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Producción,ELECTRO OUTLET SRL


In [7]:
# Recorto la columna idpais ya que no es necesaria.
avisos_detalles_total.drop(['idpais'] , axis=1, inplace=True)

In [8]:
# Agrupo las áreas parecidas.
areas_agrupadas = pd.read_csv('../extraData/areas_agrupadas.csv')
avisos_detalles_total = pd.merge(avisos_detalles_total, areas_agrupadas, how='inner', on='nombre_area')
avisos_detalles_total['nombre_area'] = avisos_detalles_total['nuevo_nombre_area']
avisos_detalles_total.drop('nuevo_nombre_area', axis=1, inplace=True)
avisos_detalles_total.head()

,idaviso,titulo,descripcion,nombre_zona,ciudad,mapacalle,tipo_de_trabajo,nivel_laboral,nombre_area,denominacion_empresa
0,8725750,VENDEDOR/A PROVINCIA DE SANTA FE,<p><strong><strong>Empresa:</strong></strong> ...,Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Comercial y Ventas,VENTOR
1,9240880,"Productores Asesores Independiente, para venta...",Agente\r\noficial Selecciona:</span></strong><...,Capital Federal,Buenos Aires,NaN,Full-time,Jefe / Supervisor / Responsable,Comercial y Ventas,Agencia Oficial Alejandro Arizaga
2,1002448818,Ejecutivo de Cuentas Sr.,<p>Importante empresa del Mercado Asegurador A...,Gran Buenos Aires,Capital Federal,NaN,Full-time,Senior / Semi-Senior,Comercial y Ventas,Romasanta y Asociados SA
3,1110319893,Vendedores Telefónicos (Part- time),<p><strong>Estamos en busqueda de representant...,Gran Buenos Aires,NaN,NaN,Part-time,Senior / Semi-Senior,Comercial y Ventas,Directway S.A.
4,1110640622,Agente Inmobiliario para Oficinas RE/MAX en Zo...,"<p>Si te gusta deleitar a tus clientes, brinda...",Capital Federal,NaN,Gobernador Irigoyen 56,Full-time,Senior / Semi-Senior,Comercial y Ventas,RE/MAX TITANIUM


In [9]:
# Convierto el nombre del area a category.
avisos_detalles_total['nombre_area'] = avisos_detalles['nombre_area'].astype('category')

In [ ]:
# Agrego tipo categorico y modifico el tipo de dato de la fecha.
post_gen_edad_total.fechanacimiento = post_gen_edad.fechanacimiento.apply(pd.to_datetime, errors='coerce')

now = pd.Timestamp(dt.datetime.now())
pd.to_datetime(now)
post_gen_edad_total['edad'] = (now - pd.to_datetime(post_gen_edad['fechanacimiento'])).astype('<m8[Y]')

In [6]:
# Renombramos algunas columnas para unificar criterios.
vistas_total.rename(columns={"idAviso":"idaviso"}     , inplace=True)
vistas_total.rename(columns={"timestamp":"fechavista"}, inplace=True)

# Cambiar el tipo de dato de la fecha de vista.
vistas_total[['fechavista']] = vistas_total[['fechavista']].apply(pd.to_datetime)
vistas_total.head()

,idaviso,fechavista,idpostulante
0,1111780242,2018-02-23 18:38:13.187,YjVJQ6Z
1,1112263876,2018-02-23 18:38:14.296,BmVpYoR
2,1112327963,2018-02-23 18:38:14.329,wVkBzZd
3,1112318643,2018-02-23 18:38:17.921,OqmP9pv
4,1111903673,2018-02-23 18:38:18.973,DrpbXDP


## Luego de mergear los df debo quitar cols

### Para poder extraer info innecesaria y poder utilizar en el entrenamiento

In [ ]:
# Asigno puntajes a los diferentes niveles educativos encontrados para poder 
# obtener asi la educacion mas valiosa (la de nivel academico mas alto) de cada postulante.
valores_educacion = pd.DataFrame({
    'nombre': [
        'Otro',  'Otro',  'Otro',  'Secundario',  'Secundario',  'Secundario',  'Terciario/Técnico', 'Universitario',  \
        'Terciario/Técnico', 'Universitario',  'Terciario/Técnico', 'Universitario',  'Posgrado', 'Master','Posgrado', \
        'Master','Posgrado', 'Master','Doctorado','Doctorado','Doctorado'
    ],
    'estado': [
        'Abandonado', 'En Curso','Graduado','Abandonado', 'En Curso','Graduado','Abandonado', 'Abandonado', 'En Curso',\
        'En Curso','Graduado','Graduado','Abandonado', 'Abandonado', 'En Curso','En Curso','Graduado','Graduado',\
        'Abandonado', 'En Curso','Graduado'
    ],
    'valor_educacion': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21]
})

# Y le pongo las mismas categorias que ya habia encontrado previamente.
valores_educacion.nombre = valores_educacion.nombre.astype((pd.api.types.CategoricalDtype(post_educacion.nombre.cat.categories)))
valores_educacion.estado = valores_educacion.estado.astype((pd.api.types.CategoricalDtype(post_educacion.estado.cat.categories)))

In [ ]:
# Hago el join para aplicar puntajes a las educaciones de los postulantes.
post_ed_total = pd.merge(post_ed_total, valores_educacion, how='inner', left_on=['nombre','estado'], right_on = ['nombre','estado'])

In [ ]:
# Busco la maxima educacion que tiene cada postulante.
max_post_educacion = post_ed_total.groupby(['idpostulante'])['valor_educacion'].max().reset_index()

In [ ]:
# Hago el join para filtrar todas las educaciones que no me interesan, dejando solo la maxima educacion.
post_ed_total = pd.merge(post_ed_total, max_post_educacion,  how='inner', left_on=['idpostulante','valor_educacion'], right_on = ['idpostulante','valor_educacion'])


In [ ]:
post_ed_total.drop_duplicates(inplace=True)

## Ahora exporto las educaciones del postulante

In [ ]:
post_ed_total.to_csv('../data/final_data/postulantes_educacion.csv', sep=',', encoding='utf-8')

## Sigo con los proximos DF

In [ ]:
post_gen_edad_total.head()

In [ ]:
#como supongo que con la edad no necesitare la fecha de nacimiento lo dropeo.
post_gen_edad_total.drop(['fechanacimiento'], axis=1, inplace=True)
post_gen_edad_total.head()

In [ ]:
#exporto al csv
post_gen_edad_total.to_csv('../data/final_data/postulantes_genero_y_edad.csv', sep=',', encoding='utf-8')

In [ ]:
vistas_total.head()

In [7]:
vistas_total.drop_duplicates(subset=['idaviso','idpostulante'], keep='last')

,idaviso,fechavista,idpostulante
0,1111780242,2018-02-23 18:38:13.187,YjVJQ6Z
1,1112263876,2018-02-23 18:38:14.296,BmVpYoR
2,1112327963,2018-02-23 18:38:14.329,wVkBzZd
3,1112318643,2018-02-23 18:38:17.921,OqmP9pv
4,1111903673,2018-02-23 18:38:18.973,DrpbXDP
7,1112282610,2018-02-23 18:38:27.670,A3R9GWl
8,1112342544,2018-02-23 18:38:31.601,8M24rbM
10,1112350129,2018-02-23 18:38:33.441,NGAjYL
11,1112335938,2018-02-23 18:38:39.226,0zdM4Ad
12,1112343977,2018-02-23 18:38:40.951,X95NQrO


In [8]:
vistas_total.to_csv('../data/final_data/vistas.csv', sep=',', encoding='utf-8')

In [12]:
postulaciones_total.sort_values( 'idaviso', ascending=True, inplace=True)
postulaciones_total.head()

,idaviso,idpostulante,fechapostulacion
1424800,8725750,YaZbp1,2018-02-07 10:20:45
2872780,8725750,ow2WG6Z,2018-01-15 22:14:50
1492835,8725750,bOzeDjd,2018-01-15 10:18:15
3592487,8725750,lDLa0LM,2018-02-02 17:19:17
2330991,8725750,mz5DvlG,2018-01-25 12:19:28


In [14]:
postulaciones_total.to_csv('../data/final_data/postulaciones.csv', sep=',', encoding='utf-8')

In [19]:
avisos_online_total.drop_duplicates(inplace=True)

In [20]:
avisos_online_total.to_csv('../data/final_data/avisos_online.csv', sep=',', encoding='utf-8')

In [21]:
avisos_online_total

,idaviso
0,1112355872
1,1112335374
2,1112374842
3,1111984070
4,1111822480
5,1112319041
6,1112318459
7,1112318868
8,1112318903
9,1112319095


In [10]:
avisos_detalles_total.drop('mapacalle',axis=1, inplace=True)

In [11]:
avisos_detalles_total.head()

,idaviso,titulo,descripcion,nombre_zona,ciudad,tipo_de_trabajo,nivel_laboral,nombre_area,denominacion_empresa
0,8725750,VENDEDOR/A PROVINCIA DE SANTA FE,<p><strong><strong>Empresa:</strong></strong> ...,Gran Buenos Aires,NaN,Full-time,Senior / Semi-Senior,Comercial,VENTOR
1,9240880,"Productores Asesores Independiente, para venta...",Agente\r\noficial Selecciona:</span></strong><...,Capital Federal,Buenos Aires,Full-time,Jefe / Supervisor / Responsable,Salud,Agencia Oficial Alejandro Arizaga
2,1002448818,Ejecutivo de Cuentas Sr.,<p>Importante empresa del Mercado Asegurador A...,Gran Buenos Aires,Capital Federal,Full-time,Senior / Semi-Senior,Transporte,Romasanta y Asociados SA
3,1110319893,Vendedores Telefónicos (Part- time),<p><strong>Estamos en busqueda de representant...,Gran Buenos Aires,NaN,Part-time,Senior / Semi-Senior,Transporte,Directway S.A.
4,1110640622,Agente Inmobiliario para Oficinas RE/MAX en Zo...,"<p>Si te gusta deleitar a tus clientes, brinda...",Capital Federal,NaN,Full-time,Senior / Semi-Senior,Producción,RE/MAX TITANIUM


In [12]:
avisos_detalles_total.to_csv('../data/final_data/avisos_detalles.csv', sep=',', encoding='utf-8')